In [ ]:
import os

In [ ]:
!rm -r AMD_ML_Chromatin/
!git clone https://github.com/ed29rice/AMD_ML_Chromatin.git
!pip install bayesian-optimization

Cloning into 'AMD_ML_Chromatin'...
remote: Enumerating objects: 257, done.
remote: Total 257 (delta 0), reused 0 (delta 0), pack-reused 257 (from 1)
Receiving objects: 100% (257/257), 3.23 MiB | 15.19 MiB/s, done.
Resolving deltas: 100% (84/84), done.


In [ ]:
import numpy as np
import math
from tempfile import TemporaryDirectory
import copy
import time
from typing import Tuple
import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset
from torch.nn import functional as F
import pickle
import matplotlib.pyplot as plt

In [ ]:
%load_ext autoreload
%autoreload 2
import AMD_ML_Chromatin.TECSAS as TECSAS

In [ ]:
NEXP = 15
n_neigbors = 2
n_predict = 1

In [ ]:
with open('./inputs/cont_chrom_odd_HR_GM12878_hg19.pickle', 'rb') as handle:
    data = pickle.load(handle)

train_set,val_set,test_set,all_averages,loci_indx = data

all_loci = np.array(list(range(len(all_averages[0]))))
ntrain_loci = all_loci[loci_indx]
nval_loci = all_loci[~loci_indx][::2]
ntest_loci = all_loci[~loci_indx][1::2]
train_data=torch.tensor(train_set,dtype=torch.float)
val_data=torch.tensor(val_set,dtype=torch.float)
test_data=torch.tensor(test_set,dtype=torch.float)
print(train_data.size(),val_data.size(),test_data.size())

torch.Size([26138, 76]) torch.Size([13153, 76]) torch.Size([13153, 76])


In [ ]:
torch.cuda.empty_cache()
nfeatures = train_data.size()[1]-(2*(n_predict-1)+1)
ostates = 5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.CrossEntropyLoss()
nbatches = 50
bptt = len(train_data)//nbatches
print('Size of batch:',bptt)

Size of batch: 522


In [ ]:
def train_model(model,ext,epochs,optimizer,scheduler):
    criterion = nn.CrossEntropyLoss()
    best_val_loss = float('inf')
    best_train_loss = float('inf')
    bsteps = 0
    for epoch in range(1, epochs + 1):
        print('Epoch:'+str(epoch),end='\r')
        lr_e = scheduler.get_last_lr()[0]
        epoch_start_time = time.time()
        train_loss=train(model,optimizer)
        elapsed = time.time() - epoch_start_time
        val_loss = evaluate(model, val_data, type='val')
        if train_loss < best_train_loss:
            best_train_loss = train_loss
            torch.save(model.state_dict(), './best_train_model_params_all'+ext+'.pt')
            bsteps = 0
        else:
            bsteps=bsteps+1
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), './best_val_model_params_all'+ext+'.pt')
        if bsteps>5:
            scheduler.step()
            bsteps=0

def train(model: nn.Module, optimizer) -> None:
    model.train()  # turn on train mode
    total_loss = 0.
    start_time = time.time()
    src_mask = None

    num_batches = len(train_data) // bptt
    bs=list(range(nbatches))
    np.random.shuffle(bs)
    counter=0
    for batch in bs:
        counter+=1
        i=batch
        data, targets = get_batch(train_data, i, n_predict=n_predict)
        seq_len = data.size(0)
        output, output_tf = model(data, src_mask)
        loss = 0
        for n in range(2*(n_predict-1)+1):
            loss += criterion(output[:,n], targets[:,n].type(torch.LongTensor).to(device))
        data.detach()
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
        total_loss = total_loss + loss.item()
    return total_loss

def evaluate(model: nn.Module, eval_data: Tensor, type='test') -> float:
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    src_mask = None
    nbatches_eval=len(eval_data)//bptt
    with torch.no_grad():
        for batch in range(nbatches_eval):
            data, targets = get_batch(eval_data, batch, n_predict=n_predict)
            output, output_tf = model(data, src_mask)
            for n in range(2*(n_predict-1)+1):
                total_loss += criterion(output[:,n], targets[:,n].type(torch.LongTensor).to(device)).item()
    return total_loss

def results(model,ext):
    k=0
    f=0
    p='best_val'
    model.load_state_dict(torch.load('./'+p+'_model_params_all'+ext+'.pt'))
    model.eval()
    nbatches_eval=len(test_data)//bptt
    l=[]
    lt=[]
    failed_inputs=[]
    failed_targets=[]
    failed_pred=[]
    failed_loci=[]
    suc_inputs=[]
    suc_targets=[]
    suc_pred=[]
    suc_loci=[]
    with torch.no_grad():
        for batch in range(nbatches_eval):
            data, targets, batch_loci = get_batch_test(test_data, batch, n_predict=n_predict, ndxs = ntest_loci)
            prediction=model(data,None)[0].argmax(dim=-1)[:,n_predict-1].cpu()
            idx=prediction!=targets[:,n_predict-1].cpu()
            failed_inputs.append(targets[idx,n_predict-1].cpu())
            failed_targets.append(data[idx].cpu())
            failed_pred.append(prediction[idx])
            failed_loci.append(batch_loci[idx])
            idx=prediction==targets[:,n_predict-1].cpu()
            suc_inputs.append(targets[idx,n_predict-1].cpu())
            suc_targets.append(data[idx].cpu())
            suc_pred.append(prediction[idx])
            suc_loci.append(batch_loci[idx])
            l.append(prediction)
            lt.append(targets[:,n_predict-1].cpu())
    failed_inputs=np.concatenate(failed_inputs)
    failed_pred=np.concatenate(failed_pred)
    failed_targets=np.concatenate(failed_targets)
    failed_loci=np.concatenate(failed_loci)
    suc_inputs=np.concatenate(suc_inputs)
    suc_pred=np.concatenate(suc_pred)
    suc_targets=np.concatenate(suc_targets)
    suc_loci=np.concatenate(suc_loci)
    with open('test.npy', 'wb') as f:
        for ppp in [failed_inputs,failed_pred,failed_targets,failed_loci,suc_inputs,suc_pred,suc_targets,suc_loci]:
            np.save(f, ppp)
        np.save(f,all_averages)
        np.save(f,loci_indx)
    l=np.concatenate(l)
    lt=np.concatenate(lt)

    conf_matrix_P=np.zeros((5,5))
    int_types_Or=lt
    types_pyME=l
    for i in range(5):
        idx1=(int_types_Or==i)
        subav=int_types_Or[idx1]
        subprd_P=types_pyME[idx1]
        for k in range(len(subprd_P)):
            for j in range(5):
                conf_matrix_P[i,j]+=(subprd_P[k]==j)
        conf_matrix_P[i,:]=np.round(conf_matrix_P[i,:]/np.sum(idx1),3)
    return np.round(np.sum(l==lt)/len(l),3), conf_matrix_P, suc_loci, failed_loci

def black_box_function(emsize_by_nhead, nhead, d_hid, nlayers, dropout, lr):
    nhead=2*int(nhead)
    emsize=int(int(emsize_by_nhead)*nhead)
    d_hid=2*int(d_hid)
    nlayers=int(nlayers)
    print(emsize_by_nhead, nhead, d_hid, nlayers, dropout, lr)
    train_data.size()[1]-(2*(n_predict-1)+1)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = TECSAS.TECSAS(1, emsize, nhead, d_hid, nlayers, nfeatures, 5, dropout).to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=lr,weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.5)
    train_model(model,'0',75,optimizer,scheduler)
    acc,conf_matrix_P,suc_loci,failed_loci=results(model,'0')
    return acc

def get_batch(source: Tensor, i: int, n_predict: int) -> Tuple[Tensor, Tensor]:
    data = source[i*bptt:(i+1)*bptt,2*(n_predict-1)+1:][:,:,np.newaxis]
    target = source[i*bptt:(i+1)*bptt,:2*(n_predict-1)+1]
    return data.to(device), target.to(device)

def get_batch_test(source: Tensor, i: int, n_predict: int, ndxs ) -> Tuple[Tensor, Tensor]:
    data = source[i*bptt:(i+1)*bptt,2*(n_predict-1)+1:][:,:,np.newaxis]
    target = source[i*bptt:(i+1)*bptt,:2*(n_predict-1)+1]
    indexes = ndxs[i*bptt:(i+1)*bptt]
    return data.to(device), target.to(device), indexes

In [ ]:
# Bounded region of parameter space
pbounds = {'emsize_by_nhead':(2,16), 'nhead':(1,4), 'd_hid':(1,64), 'nlayers':(1,4), 'dropout':(0.01,0.9), 'lr':(1e-2,2)}
# Embedding dimension = emsize_by_head*2*nhead
# Number of heads = 2*nhead
# Hidden layer dimension = 2*d_hid

In [ ]:
from bayes_opt import BayesianOptimization

optimizer_bo = BayesianOptimization(
    f=black_box_function,
    pbounds=pbounds,
)

In [ ]:
optimizer_bo.set_gp_params(alpha=1e-2)
optimizer_bo.maximize(init_points=5, n_iter=35)

In [ ]:
!awk '{print $1,$2,$1,$4,$1,$16,$1,$6,$1,$14,$1,$10,$1,$12,$1,$8,$1}' dummy2 | head -n1
!awk '{print $1,$2,$1,$4,$1,int($16),$1,2*int($6),$1,2*int($14),$1,int($10)*2*int($14),$1,$12,$1,$8,$1}' dummy2 | tail -n+2

| iter | target | nlayers | d_hid | nhead | emsize... | lr | dropout |
| 1 | 0.544 | 3 | 24 | 2 | 12 | 0.6077 | 0.8681 |
| 2 | 0.601 | 3 | 120 | 2 | 18 | 1.835 | 0.323 |
| 3 | 0.594 | 3 | 56 | 4 | 24 | 0.9001 | 0.433 |
| 4 | 0.566 | 2 | 82 | 2 | 14 | 0.1487 | 0.2527 |
| 5 | 0.579 | 2 | 54 | 6 | 12 | 0.9097 | 0.7364 |
| 6 | 0.603 | 3 | 126 | 2 | 24 | 0.789 | 0.03329 |
| 7 | 0.6 | 1 | 114 | 6 | 90 | 1.577 | 0.04214 |
| 8 | 0.583 | 1 | 126 | 6 | 12 | 1.698 | 0.4483 |
| 9 | 0.584 | 3 | 56 | 2 | 30 | 0.4765 | 0.306 |
| 10 | 0.573 | 3 | 104 | 2 | 30 | 0.7702 | 0.4007 |
| 11 | 0.557 | 1 | 122 | 6 | 72 | 0.1007 | 0.382 |
| 12 | 0.588 | 2 | 58 | 4 | 8 | 0.5022 | 0.09446 |
| 13 | 0.562 | 1 | 38 | 6 | 90 | 1.479 | 0.8044 |
| 14 | 0.587 | 1 | 98 | 6 | 78 | 0.1926 | 0.4081 |
| 15 | 0.581 | 2 | 110 | 2 | 24 | 1.439 | 0.2504 |
| 16 | 0.568 | 3 | 114 | 4 | 16 | 1.053 | 0.6968 |
| 17 | 0.58 | 3 | 56 | 2 | 30 | 0.515 | 0.3136 |
| 18 | 0.55 | 3 | 126 | 2 | 26 | 1.824 | 0.7959 |
| 19 | 0.588 | 3 | 118 | 2

In [ ]:
iterations = [0.544,0.601,0.594,0.566,0.579,0.603,0.6  ,0.583,0.584,0.573,0.557,0.588,0.562,0.587,0.581,0.568,0.58 ,0.55 ,0.588,0.573,0.546,0.604,0.565,0.602,0.605,0.562,0.592,0.606,0.593,0.583,0.573,0.581,0.584,0.542,0.591]
plt.plot(iterations,'.-',markersize=10)
plt.title('Bayesian Optimization Progress')
plt.xlabel('Iteration')
plt.ylabel('Accuracy')
plt.grid()